# Seleção de Modelo e Regressão 
---

## AIC

---

Em alguns problemas, podem-se usar muitas variáveis como preditoras em uma regressão. No entanto, adicionar mais variáveis não significa necessariamente que teremos um modelo melhor. Os estatísticos usam o princípio da _navalha de Occam_  para guiar a escolha de um modelo. 

Incluir variáveis adicionar sempre reduz o RMSE e aumenta o R^2 como observado na métrica AIC (Akaike's Information Criteria). 

No contexto da **regressão linear**, ao ajustar modelos diferentes, o AIC ajuda a escolher aquele que melhor equilibra a complexidade e o ajuste aos dados. Modelos com valores mais baixos de AIC são preferíveis, pois indicam um melhor ajuste relativo, penalizando menos pela complexidade.

AIC = 2P+nlog(RSS/n)

In [3]:
import statsmodels.api as sm
import numpy as np

# Dados de exemplo
X = np.random.rand(100, 3)
y = np.random.rand(100)

# Adicionar constante (intercepto) ao modelo
X = sm.add_constant(X)

# Ajustar o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Exibir o valor do AIC
print('AIC:', modelo.aic)



AIC: 54.55250860629448


### **Seleção Progressiva (Forward Selection)**
---

A seleção progressiva começa com um modelo simples, sem variáveis, e vai adicionando variáveis uma a uma. A cada passo, a variável que mais melhora o critério de ajuste (como AIC, BIC, ou R2) é incluída no modelo.

**Passos:**

*   Começa sem variáveis no modelo.
*   Adiciona a variável que mais melhora o critério de ajuste.
*   Repete o processo até que nenhuma variável adicional melhore significativamente o critério.

In [5]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Gerar dados de exemplo
np.random.seed(42)
X = pd.DataFrame(np.random.rand(100, 5), columns=['var1', 'var2', 'var3', 'var4', 'var5'])
y = pd.Series(np.random.rand(100))

# Adicionar constante ao modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Função de Seleção Progressiva (Forward Selection)
def forward_selection(X, y):
    initial_features = []
    remaining_features = list(X.columns)
    selected_features = []
    best_aic = np.inf
    
    while remaining_features:
        aics_with_candidates = []
        
        for candidate in remaining_features:
            # Modelo com as features já selecionadas + candidata
            features = initial_features + [candidate]
            X_with_candidate = sm.add_constant(X[features])
            model = sm.OLS(y, X_with_candidate).fit()
            aics_with_candidates.append((model.aic, candidate))
        
        # Seleciona o modelo com menor AIC
        aic, best_candidate = min(aics_with_candidates, key=lambda x: x[0])
        
        if aic < best_aic:
            best_aic = aic
            initial_features.append(best_candidate)
            remaining_features.remove(best_candidate)
            selected_features.append(best_candidate)
        else:
            break
    
    return selected_features

# Executar Seleção Progressiva
best_features = forward_selection(X_train, y_train)

# Exibir as melhores variáveis selecionadas
print("Variáveis Selecionadas:", best_features)

# Ajustar o modelo final com as variáveis selecionadas
X_selected = sm.add_constant(X_train[best_features])
modelo_final = sm.OLS(y_train, X_selected).fit()

# Exibir o sumário do modelo final
print(modelo_final.summary())


Variáveis Selecionadas: ['var4', 'var1', 'var5']
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.168
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     5.112
Date:                Wed, 02 Oct 2024   Prob (F-statistic):            0.00283
Time:                        22:26:52   Log-Likelihood:                -9.0609
No. Observations:                  80   AIC:                             26.12
Df Residuals:                      76   BIC:                             35.65
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
con

#### Explicação:

forward_selection: Essa função itera sobre as variáveis disponíveis, adicionando-as ao modelo uma de cada vez e calculando o AIC. A cada iteração, a variável que mais reduz o AIC é mantida, até que nenhuma adição de variável melhore o modelo.

Modelo Final: Após a seleção progressiva, o modelo final é ajustado com as variáveis selecionadas e o sumário estatístico é exibido.

Notas:

O critério de seleção usado aqui é o AIC.

A função seleciona variáveis até que nenhuma outra adição melhore o valor do AIC.

### **Seleção Regressiva (Backward Elimination)**
---

Na seleção regressiva, o processo é o oposto: começa-se com todas as variáveis no modelo, e a cada passo, remove-se a variável menos significativa, ou aquela que mais prejudica o critério de ajuste.

**Passos:**

*   Começa com todas as variáveis no modelo.
*   Remove a variável menos significativa (maior p-valor ou que piora o critério de ajuste).
*   Repete o processo até que nenhuma variável possa ser removida sem piorar significativamente o critério.

In [6]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Gerar dados de exemplo
np.random.seed(42)
X = pd.DataFrame(np.random.rand(100, 5), columns=['var1', 'var2', 'var3', 'var4', 'var5'])
y = pd.Series(np.random.rand(100))

# Adicionar constante ao modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Função de Seleção Regressiva (Backward Elimination)
def backward_elimination(X, y, significance_level=0.05):
    features = list(X.columns)
    
    while len(features) > 0:
        X_with_constant = sm.add_constant(X[features])
        model = sm.OLS(y, X_with_constant).fit()
        p_values = model.pvalues.iloc[1:]  # Excluir o p-valor do intercepto (const)
        max_p_value = p_values.max()
        
        if max_p_value > significance_level:
            # Remove a variável com o maior p-valor
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
            print(f"Removendo variável {excluded_feature} com p-valor {max_p_value}")
        else:
            break
    
    return features

# Executar Seleção Regressiva
best_features = backward_elimination(X_train, y_train)

# Exibir as melhores variáveis selecionadas
print("Variáveis Selecionadas:", best_features)

# Ajustar o modelo final com as variáveis selecionadas
X_selected = sm.add_constant(X_train[best_features])
modelo_final = sm.OLS(y_train, X_selected).fit()

# Exibir o sumário do modelo final
print(modelo_final.summary())


Removendo variável var3 com p-valor 0.8251818703813887
Removendo variável var2 com p-valor 0.45466654280745067
Removendo variável var5 com p-valor 0.07460578207405656
Removendo variável var1 com p-valor 0.07584286597619241
Variáveis Selecionadas: ['var4']
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     8.247
Date:                Wed, 02 Oct 2024   Prob (F-statistic):            0.00525
Time:                        22:29:05   Log-Likelihood:                -12.393
No. Observations:                  80   AIC:                             28.79
Df Residuals:                      78   BIC:                             33.55
Df Model:                           1                                         
Covariance Type:            nonro

#### Explicação:

backward_elimination: A função ajusta um modelo de regressão linear com todas as variáveis e, em seguida, remove a variável com o maior p-valor (menos significativa). O processo continua até que todas as variáveis restantes tenham um p-valor menor que o nível de significância fornecido (neste caso, 0.05).

Critério de exclusão: A variável com o maior p-valor é removida em cada iteração, até que todas as variáveis no modelo sejam estatisticamente significativas.

Modelo Final: Depois da eliminação regressiva, o modelo final é ajustado com as variáveis restantes e o sumário estatístico do modelo é exibido.

Resultado:

Durante a execução, o código exibirá as variáveis sendo removidas e, no final, o modelo será ajustado apenas com as variáveis selecionadas.

Esse método é útil para remover variáveis irrelevantes e melhorar a interpretabilidade do modelo.

### Regressão Penalizada
---

A regressão penalizada é uma técnica mais sofisticada que envolve a adição de um termo de penalidade à função de ajuste, para evitar overfitting e selecionar variáveis automaticamente. As formas mais comuns de regressão penalizada são **Ridge** e **Lasso**.

- **Ridge Regression** adiciona uma penalidade proporcional ao quadrado dos coeficientes. Ela reduz os coeficientes, mas não os zera. A fórmula penalizada da função de custo é:

 $$\text{Custo Ridge} = \sum (y_i - \hat{y_i})^2 + \lambda \sum \beta_j^2$$

- **Lasso Regression** (Least Absolute Shrinkage and Selection Operator) adiciona uma penalidade proporcional ao valor absoluto dos coeficientes. Além de reduzir, o Lasso pode zerar alguns coeficientes, funcionando como uma técnica de seleção de variáveis:

 $$\text{Custo Lasso} = \sum (y_i - \hat{y_i})^2 + \lambda \sum |\beta_j|$$

 **Elastic Net** combina as penalidades do Ridge e do Lasso, dando um controle maior sobre a seleção de variáveis e a regularização.

In [7]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

# Gerar dados de exemplo
np.random.seed(42)
X = pd.DataFrame(np.random.rand(100, 5), columns=['var1', 'var2', 'var3', 'var4', 'var5'])
y = pd.Series(np.random.rand(100))

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar o modelo Lasso
modelo_lasso = Lasso(alpha=0.1)  # 'alpha' é o parâmetro de regularização
modelo_lasso.fit(X_train, y_train)

# Predição e avaliação
y_pred = modelo_lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print('MSE (Lasso):', mse)
print('Coeficientes (Lasso):', modelo_lasso.coef_)


MSE (Lasso): 0.08586156548817339
Coeficientes (Lasso): [-0. -0. -0.  0. -0.]


#### Explicação:

1.  **Lasso Regression**: O modelo de regressão Lasso é ajustado utilizando o parâmetro `alpha` que controla a força da regularização. Um valor maior de `alpha` penaliza mais os coeficientes, podendo forçar alguns coeficientes a zero, o que implica em uma seleção de variáveis.
2.  **Treinamento e Teste**: Os dados são divididos em conjuntos de treinamento e teste para avaliar o modelo.
3.  **Coeficientes**: O Lasso pode zerar alguns coeficientes, permitindo que variáveis irrelevantes sejam eliminadas do modelo.
4.  **Erro Quadrático Médio (MSE)**: O erro do modelo é calculado com base no conjunto de teste.

### Notas:

*   O parâmetro `alpha` controla o quanto a penalização impacta o modelo. Valores maiores de `alpha` resultam em mais variáveis sendo eliminadas (coeficientes sendo zerados).
*   O `coef_` do modelo exibirá os coeficientes de cada variável após a regularização. Coeficientes zero indicam que a variável foi eliminada pelo Lasso.

In [8]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

# Gerar dados de exemplo
np.random.seed(42)
X = pd.DataFrame(np.random.rand(100, 5), columns=['var1', 'var2', 'var3', 'var4', 'var5'])
y = pd.Series(np.random.rand(100))

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar o modelo Ridge
modelo_ridge = Ridge(alpha=0.1)  # 'alpha' é o parâmetro de regularização
modelo_ridge.fit(X_train, y_train)

# Predição e avaliação
y_pred = modelo_ridge.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print('MSE (Ridge):', mse)
print('Coeficientes (Ridge):', modelo_ridge.coef_)


MSE (Ridge): 0.12296817407252772
Coeficientes (Ridge): [-0.21544454 -0.07665229 -0.02477832  0.36864494 -0.18638862]


#### Explicação:

1.  **Ridge Regression**: A técnica de **Ridge** é usada para penalizar os coeficientes grandes, adicionando uma penalidade proporcional ao quadrado dos coeficientes:

   $$
   \sum \beta_j^2
   $$

   Isso ajuda a reduzir o overfitting.

2.  **Parâmetro `alpha`**: O parâmetro `alpha` controla a intensidade da penalização. Quanto maior o valor de `alpha`, maior a penalização, o que tende a reduzir ainda mais os coeficientes.

3.  **Divisão dos Dados**: Os dados são divididos em conjuntos de treinamento e teste. O modelo é treinado com o conjunto de treinamento e avaliado com o conjunto de teste.

4.  **Erro Quadrático Médio (MSE)**: O erro do modelo é calculado no conjunto de teste para avaliar o desempenho.

5.  **Coeficientes**: Os coeficientes do modelo Ridge são exibidos. A penalização pode reduzir os coeficientes em magnitude, mas normalmente não os zera completamente como no Lasso.

### Notas:

- O valor de `alpha` deve ser ajustado com base na quantidade de regularização desejada. Um valor muito pequeno pode resultar em overfitting, enquanto um valor muito grande pode prejudicar o ajuste ao ponto de ignorar quase todas as variáveis.
